In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df=pd.read_csv(r'C:\Users\SAYAN ROY\OneDrive\Documents\Datascience\new datasets\train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.dropna(inplace=True)

In [4]:
X=df.drop('label',axis=1)
y=df.label

In [5]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense 

In [6]:
voc_size=5000
message=X.copy()
message.reset_index(inplace=True)

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [8]:
ps=PorterStemmer()
corpus=[]
for i in range(0,len(message)):
    rev=re.sub('[^a-zA-Z]',' ',message['title'][i])
    rev=rev.lower()
    rev=rev.split()
    rev=[ps.stem(word) for word in rev if not word in stopwords.words('english')]
    rev=' '.join(rev)
    corpus.append(rev)

In [9]:
one_hot_rep = [one_hot(words,voc_size) for words in corpus]
one_hot_rep[:5]

[[3129, 3398, 1463, 74, 96, 3174, 3946, 2596, 305, 4517],
 [1700, 3904, 785, 745, 3574, 2342, 914],
 [1335, 1706, 480, 1136],
 [3783, 1096, 1004, 3133, 1468, 1327],
 [4555, 3574, 1383, 249, 1771, 3329, 3574, 1087, 4497, 3260]]

In [10]:
max_len=20
embedded_doc=pad_sequences(one_hot_rep,padding='pre',maxlen=max_len)
embedded_doc[:4]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3129,
        3398, 1463,   74,   96, 3174, 3946, 2596,  305, 4517],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 1700, 3904,  785,  745, 3574, 2342,  914],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 1335, 1706,  480, 1136],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 3783, 1096, 1004, 3133, 1468, 1327]])

In [11]:
embedding_vector=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector,input_length=max_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
import numpy as np
X_final=np.array(embedded_doc)
y_final=np.array(y)

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.3,random_state=42)

In [14]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=32)

Epoch 1/100
400/400 [==============================] - 9s 15ms/step - loss: 0.2700 - accuracy: 0.8786 - val_loss: 0.1937 - val_accuracy: 0.9185
Epoch 2/100
400/400 [==============================] - 6s 14ms/step - loss: 0.1396 - accuracy: 0.9453 - val_loss: 0.1950 - val_accuracy: 0.9185
Epoch 3/100
400/400 [==============================] - 6s 14ms/step - loss: 0.1000 - accuracy: 0.9642 - val_loss: 0.2287 - val_accuracy: 0.9216
Epoch 4/100
400/400 [==============================] - 6s 14ms/step - loss: 0.0697 - accuracy: 0.9745 - val_loss: 0.2365 - val_accuracy: 0.9182
Epoch 5/100
400/400 [==============================] - 6s 14ms/step - loss: 0.0502 - accuracy: 0.9827 - val_loss: 0.2729 - val_accuracy: 0.9149
Epoch 6/100
400/400 [==============================] - 6s 14ms/step - loss: 0.0353 - accuracy: 0.9884 - val_loss: 0.3228 - val_accuracy: 0.9100
Epoch 7/100
400/400 [==============================] - 5s 13ms/step - loss: 0.0265 - accuracy: 0.9916 - val_loss: 0.3652 - val_accuracy:

In [22]:
thrs=0.5
y_pred=(model.predict(X_test) > thrs).astype(int)

172/172 [==============================] - 1s 4ms/step


In [23]:
from sklearn.metrics import confusion_matrix,accuracy_score
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[2828  279]
 [ 196 2183]]
0.9134159679183376
